In [ ]:
import json
import os
from pathlib import Path

### CONVERT FILELIST

In [ ]:
#TODO: Convert rawframe filelist to video filelist -remove the total_frames from .txt line 
#WARNING: use it if you sure that all the videos were converted
dataset='INCAR'
dataset_type='videos' 
datadir='/home/administrator/Z/Algorithms/mmaction2/data/INCAR/'
outdir=datadir

a_file=open(datadir+f'{dataset}_total_rawframes.txt','r')
filelist=[]
for l in a_file:
    strip_line=l.strip()
    line_list=strip_line.split()
    del line_list[-2]
    line_list[0]=line_list[0]+'.mp4'
    line=' '.join([ll for ll in line_list])+'\n'
    filelist.append(line)
a_file.close() 
#print(filelist)
    
   
    
filename=f'{dataset}_total1_{dataset_type}.txt'
with open(outdir + filename, 'w') as f:
    f.writelines(filelist)

### GENERATE FILELIST

In [ ]:
classdir=os.getcwd()
cls2id = json.load(open(os.path.join(classdir,'cls2id.txt')))
id2cls = json.load(open(os.path.join(classdir,'id2cls.txt')))
correctedDict = {}
for key, value in id2cls.items():
	try:
		key = int(key)
	except Exception as ex:
		pass
	correctedDict[key] = value
id2cls = correctedDict #id2cls needs to be int keys

In [ ]:
def list_files(startpath, method="videos", level=2):
    #WARNING: For now only works for level 2; 
    #TODO: For level 1, the annotation json file is needed to extract video and rawframe labels
    if level==1: raise 
    
    filelist=[]
    first=True
    for root, dirs, files in os.walk(startpath):
        if first: #don't use the first root
            first=False
            continue 
        label=''
        if level==2: 
            category=os.path.basename(root)
            label=cls2id[category]
        if method=="videos":
            for f in files:
                # video annotation file list: json to txt [video_path label]
                video_fn=f
                vidline = f'{video_fn} {label}\n'
                if level==2: vidline = f'{category}/{video_fn} {label}\n' 
                filelist.append(vidline) 
        if method=="rawframes": 
            for d in dirs:
                # rawframe annotation file list: json to txt [frame_directory total_frames label]
                video_fn=d
                total_frames=len(files)
                frameline = f'{video_fn} {total_frames} {label}\n'
                if level==2: vidline = f'{category}/{video_fn} {total_frames} {label}\n' 
                filelist.append(frameline) 
    return filelist
#dataset_type='videos' #
#datadir='/home/administrator/Z/Algorithms/mmaction2/data/INCAR/'
#list_files(datadir+dataset_type+'/', method=dataset_type, level=2)

In [ ]:
# GENERATE FILELISTS
#save imglist : mola_{train,val}_rawframes.txt
dataset='INCAR'
dataset_type='videos' #
datadir='/home/administrator/Z/Algorithms/mmaction2/data/INCAR/'
outdir=datadir
filelist=list_files(datadir+dataset_type+'/', method=dataset_type, level=2)

filename=f'{dataset}_total_{dataset_type}.txt'
with open(outdir + filename, 'w') as f:
    f.writelines(filelist)

### SPLIT FILELIST

In [ ]:
def split_filelist(path):
    filelist=[]
    with open(path) as f:
        filelist=f.readlines()
    split=['train','val','test']
    for s in split:
        ps=[]
        if s=="train": ps=['P'+str(i+1)+'_' for i in range(8)]
        if s=="val": ps=['P'+str(i+1)+'_' for i in range(8,12)]
        if s=="test": ps=['P'+str(i+1)+'_' for i in range(12,16)]
        filelist_split=[l for l in filelist for p in ps if l.find(p)>-1]
        filelist_split=list(dict.fromkeys(filelist_split))
        fn_split=Path(path).stem.split('_')
        fn_split[-2]=s
        fn_split='_'.join(fn_split)
        
        with open(os.path.join(os.path.dirname(path),fn_split+'.txt'), 'w') as f:
            f.writelines(filelist_split)
    return
split_filelist("/home/administrator/Z/Algorithms/mmaction2/data/INCAR/INCAR_total_videos.txt")